# <span style="color:teal">RBFE Network - Analysis</span>


In [ ]:
# import libraries

from scipy.stats import sem as sem
import sys
import glob

if "/home/anna/Documents/cinnabar" not in sys.path:
    sys.path.insert(1, "/home/anna/Documents/cinnabar")
import cinnabar

print("adding code to the pythonpath...")
code = "/home/anna/Documents/code/python"
if code not in sys.path:
    sys.path.insert(1, code)
import pipeline

print(cinnabar.__file__)

from pipeline import *
from pipeline.utils import validate
from pipeline.analysis import *

In [ ]:
bench_folder = f"/home/anna/Documents/benchmark"
protein = "mcl1"
main_dir = f"{bench_folder}/extracted/{protein}"
# main_dir = f"/backup/GROMACS_reruns/{protein}"

# choose location for the files
net_file = f"{main_dir}/execution_model/network_combined.dat"
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
exp_file = f"{bench_folder}/inputs/experimental/{protein}.yml"
results_folder = f"{main_dir}/outputs_extracted/results"
output_folder = validate.folder_path(f"{main_dir}/analysis", create=True)

In [ ]:
all_analysis_object = analysis_network(
    results_folder,
    exp_file=exp_file,
    net_file=net_file,
    output_folder=output_folder,
    analysis_prot=ana_file,
    # method = "updated",
    # engines = "GROMACS"
)

# can add any other results files
# all_analysis_object.compute_other_results(file_name=None, name=None)
all_analysis_object.compute_results()

In [ ]:
all_analysis_object.add_ligands_folder(f"/home/anna/Documents/benchmark/inputs/{protein}/ligands")
all_analysis_object._initialise_graph_object()
# all_analysis_object.compute_cycle_closures()
cycles = all_analysis_object.network_graph.cycle_closures()

In [ ]:
all_analysis_object.calc_pert_dict["GROMACS"]

In [ ]:
all_analysis_object.perturbing_atoms_and_overlap(prep_dir=f"/home/anna/Documents/benchmark/{protein}_benchmark/prep", outputs_dir=f"{main_dir}/outputs_extracted")

In [ ]:
# check if there are any failed perturbations
failed_perts = all_analysis_object.failed_runs("GROMACS")
# print(failed_perts)
for pert in sorted(failed_perts):
    print(pert)

In [ ]:
# all_analysis_object._initialise_graph_object(ligands_folder=f"{bench_folder}/inputs/{protein}/ligands")
# all_analysis_object.draw_failed_perturbations(eng="GROMACS")

In [ ]:
all_analysis_object.disconnected_ligands(eng="GROMACS")

In [ ]:
all_analysis_object.draw_graph()

In [ ]:
# can compute convergence for all
all_analysis_object.compute_convergence(main_dir=main_dir)
all_analysis_object.plot_convergence()

There are different options for plotting:
bar (pert or lig)
scatter (pert or lig) - can be plotted using cinnabar
eng vs eng
other results (eg from other workflows)
outliers
histograms (for the error of the runs, sem, legs)

In [ ]:
all_analysis_object.plot_scatter_pert()

In [ ]:
# bar
all_analysis_object.plot_bar_lig()
all_analysis_object.plot_bar_pert()

# scatter
all_analysis_object.plot_scatter_lig()
all_analysis_object.plot_scatter_pert()
all_analysis_object.plot_scatter_lig(use_cinnabar=True)
all_analysis_object.plot_scatter_pert(use_cinnabar=True)

for eng in all_analysis_object.engines:
    all_analysis_object.plot_scatter_lig(engine=eng)
    all_analysis_object.plot_scatter_pert(engine=eng)

    # outliers
    all_analysis_object.plot_outliers(engine=eng)
    all_analysis_object.plot_outliers(engine=eng, pert_val="val")

all_analysis_object.plot_histogram_legs()
all_analysis_object.plot_histogram_repeats()
all_analysis_object.plot_histogram_sem(pert_val="pert")
all_analysis_object.plot_histogram_sem(pert_val="val")

calculate the stats:

In [ ]:
for eng in ["GROMACS"]:
    rerun_folder = "/backup/GROMACS_reruns/mcl1/outputs_extracted/results"

    other_results = glob.glob(
        f"{rerun_folder}/freenrg_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    bound_results = glob.glob(
        f"{rerun_folder}/bound_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    free_results = glob.glob(
        f"{rerun_folder}/free_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
    )
    all_analysis_object.compute_other_results(
        other_results, name=f"{eng}_rerun", bound_files=bound_results, free_files=free_results
    )

In [ ]:
# remove any non main netowrk perturbations
for eng in all_analysis_object.other_results_names:
    for pert in all_analysis_object._perturbations_dict[eng]:
        if pert not in all_analysis_object._perturbations_dict["GROMACS"]:
            all_analysis_object.remove_perturbations(pert, name=eng)

In [ ]:
df = all_analysis_object.calc_mad_engines(pert_val="pert")
# all_analysis_object.calc_mad_engines(pert_val="val")
# stats_dict = all_analysis_object.calc_stats()
df

In [ ]:
df = all_analysis_object.calc_mad_engines(pert_val="pert")all_analysis_object.calc_mad_engines(pert_val="pert")
df

In [ ]:
all_analysis_object.plot_scatter_lig(engine="GROMACS", use_cinnabar=True)

In [ ]:
all_analysis_object._initialise_stats_object()
mae = all_analysis_object._stats_object.compute_mue(y="GROMACS", pert_val="val")
stats_string = f"MAE: {mae[0]:.2f} +/- {mae[1]:.2f} kcal/mol"
print(stats_string)

plotting with stats as title

In [ ]:
# MAE calculate
for eng in all_analysis_object.engines:
    mae = all_analysis_object._stats_object.compute_mue(y=eng, pert_val="pert")
    stats_string = f"MAE: {mae[0]:.2f} +/- {mae[1]:.2f} kcal/mol"
    all_analysis_object.plot_scatter_pert(
        engine=eng,
        **{"title": stats_string, "key": False,
        #    "save": f"/home/anna/Desktop/{eng}"
           },
    )

sort by binding affinity

In [ ]:
all_analysis_object.sort_ligands_by_binding_affinity(engine="SOMD")
all_analysis_object.sort_ligands_by_experimental_binding_affinity()

plot other results

In [ ]:
eng = "GROMACS"
rerun_folder = "/backup/GROMACS_reruns/tyk2/outputs_extracted/results"

other_results = glob.glob(
    f"{rerun_folder}/freenrg_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
)
bound_results = glob.glob(
    f"{rerun_folder}/bound_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
)
free_results = glob.glob(
    f"{rerun_folder}/free_*_{eng}_MBAR_alchemlyb_None_eqfalse_statsfalse_truncate0end.csv"
)
all_analysis_object.compute_other_results(
    other_results, name=f"GROMACS_reruns", bound_files=bound_results, free_files=free_results
)


In [ ]:
all_analysis_object._plotting_object._eng_other_list()
all_analysis_object._plotting_object.names_list

In [ ]:
all_analysis_object.plot_outliers(engines=["GROMACS","GROMACS_reruns"], no_outliers=3)

In [ ]:
all_analysis_object.network_graph.draw_perturbation("lig_43~lig_45")

In [ ]:
all_analysis_object.add_ligands_folder(f"/home/anna/Documents/benchmark/inputs/{protein}/ligands")
all_analysis_object._initialise_graph_object()
# all_analysis_object.draw_failed_perturbations(eng="AMBER")

In [ ]:
all_analysis_object.plot_bar_pert(engine=["SOMD"],
                                  **{"values":all_analysis_object._perturbations_dict["SOMD"]}
                                  )

In [ ]:
all_analysis_object.plot_histogram_repeats(engines=["GROMACS","GROMACS_earlier"])

remove a perturbation

In [ ]:
# check which perturbations are outliers
all_analysis_object.plot_outliers(engine="GROMACS", outliers=5, pert_val="pert")
# all_analysis_object._plotting_object.outlier(engines="AMBER", outliers=5, name="Exscientia")

In [ ]:
# remove perturbation for all engines
all_analysis_object.remove_perturbations("lig_c~lig_d")

remove outliers

In [ ]:
all_analysis_object.remove_outliers(threshold=10, verbose=True)

calculate only the experimental results for some of the perturbations

In [ ]:
protein = "mcl1"

bench_folder = f"/home/anna/Documents/benchmark"
main_dir = f"{bench_folder}/extracted/{protein}"

# choose location for the files
# net_file = f"{main_dir}/execution_model/network_lomap.dat"
ana_file = f"{main_dir}/execution_model/analysis_protocol.dat"
# exp_file = f"{bench_folder}/inputs/experimental/tyk2.yml"
exp_file = f"/home/anna/Documents/benchmark/inputs/experimental/{protein}.yml"

output_folder = validate.folder_path(f"{main_dir}/diff_methods_analysis", create=True)
results_folder = f"{main_dir}/diff_methods_analysis/results"

all_analysis_object = analysis_network(exp_file=exp_file)

all_analysis_object.perturbations = [
    "lig_27~lig_59",
    "lig_27~lig_48",
    "lig_28~lig_47",
    "lig_35~lig_52",
    "lig_38~lig_48",
    "lig_43~lig_45",
    "lig_53~lig_56",
    "lig_30~lig_40",
    "lig_27~lig_48",
]
all_analysis_object.ligands = get_ligands_from_perts(all_analysis_object.perturbations)
all_analysis_object.get_experimental()
all_analysis_object.get_experimental_pert()